# Analyse slip velocity particle
In this notebook I analyse the slip velocity (difference fluid velocity and particle velocity)   
to make an estimation for the Reynolds partilcle number ($Re_p = U_{slip} d / \nu $). As  
pointed out by Irina, $Re_p$ might not be small and thus additional corrections to the Stokes  
drag force in the form of $\nu\rightarrow\nu_{eff}=\nu c(Re_p) = \nu(1+f(RE_p))$ should be added.       
To investigate this we measure the slip velocity in simulations with MR advection using:  
1. stokes drag
2. drag with  $c(Re_p)$ dependend drag   
And we do this for the slow manifold and full MR equation. We also investigate:   
1. variability $Re_p$
2. spin up time simulation (how mcuh time is needed before  slip velocity / $Re_p$ stabilizes)
32. effect of different timesteps


In [ ]:
# import needed packages

#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import Divider, Size
import matplotlib.cm as cm
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from decimal import Decimal
from scipy.optimize import fsolve
from matplotlib import colormaps

from datetime import datetime, timedelta
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions_xr import trajectory_length, Haversine_list, calc_tidal_av
from analysis_functions import make_PDF, make_lognormal_PDF
import particle_characteristics_functions as pcf

sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/simulations")
from helper import create_filelist

plt.style.use('../python_style_Meike.mplstyle')
# plotstyle:
color_array = np.array(['c','orange','purple','black'])
marker_array = np.array(['s','o','>','p','^','D'])
line_array = np.array(['-','--','-.',':'])
markerline_array = np.array(['--s','--o','-->','--p','--^','--D'])
Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.0, 0.9, 6))

In [ ]:
# set needed constants
Rearth = 6371 * 10**3 # in m,
deg2rad = np.pi / 180.
sec_in_hours= 3600
diameter = 0.2 #m
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
tau = 3196 #2994.76 #2759.97
av_temp_NWES = 11.983276
av_salinity_NWES = 34.70449
rho_water = 1027 # kg/m3 
dynamic_viscosity_water = pcf.dynamic_viscosity_Sharqawy(av_temp_NWES,av_salinity_NWES/1000)# 1.e-3# 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water

## 48 hours sim

In [ ]:
data_path = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
basefile_Rep_constant = data_path+'{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4__Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr'
basefile_Rep_flexible = data_path+'{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr'

In [ ]:
# read in data

#settings
loc = 'NWES'
land_handling = 'anti_beaching'
B = 0.68
tau = 3196# 2994.76# 2759.97
coriolis = True
gradient = True
chunck_time =100
nparticles = 52511

starttime = datetime(2023, 9, 1, 0, 0, 0, 0)
runtime =  timedelta(days=2)# timedelta(days=10)
endtime = starttime + runtime

particle_types = ['inertial_Rep_constant','inertial_drag_Rep']
Replist= np.array([0,200,400,600,800,1000])
Repvalues = {'inertial_Rep_constant':Replist,
             'inertial_drag_Rep':[None]}
filenames = {'inertial_Rep_constant':basefile_Rep_constant,
             'inertial_drag_Rep':basefile_Rep_flexible}

data = {}
for pt in particle_types:
    data[pt]={}

for pt in particle_types:
    for Rep in Repvalues[pt]: 
        file = filenames[pt].format(loc=loc,
                                    particle_type = pt,
                                    y_s=starttime.year,
                                    m_s=starttime.month,
                                    d_s=starttime.day,
                                    y_e=endtime.year,
                                    m_e=endtime.month,
                                    d_e=endtime.day,
                                    B = int(B * 1000), 
                                    tau = int(tau ),
                                    land_handling = land_handling, 
                                    coriolis = coriolis,
                                    Rep = Rep ,
                                    gradient = gradient)
        ds = xr.open_dataset(file,
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=False) #,decode_cf=False)
        Uslip = np.sqrt(ds.uslip**2 + ds.vslip**2) 
        ds = ds.assign(Uslip = Uslip)
        Rep_measured = pcf.Re_particle(Uslip,diameter, kinematic_viscosity_water)
        ds = ds.assign(Rep_measured = Rep_measured)
        ds.Uslip[:,0]=np.nan
        data[pt][Rep]=ds
    

### measured Rep as function of time for 1 particle

In [ ]:
# plot selection of slip velocities c(Rep=0) is not plotted as this is way bigger
id=25
Replist_selection = Replist#[200,450,600,800,1000]
legend = []
pt = 'inertial_Rep_constant'


fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))

for Rep, color in zip(Replist_selection, Magmalist):
    if Rep ==0:
        continue
    ax.plot((data[pt][Rep].time[id,1::12]-data[pt][Rep].time[id,0])/sec_in_hours,data[pt][Rep].Rep_measured[id,1::12],'-.o',color=color)#,marker_array[i],color=color_array[i])   
    legend.append(f'c({Rep}) = {pcf.factor_drag_white1991(Rep):.0f}')



pt = 'inertial_drag_Rep'
Rep = None
ax.plot((data[pt][Rep].time[id,1::12]-data[pt][Rep].time[id,0])/sec_in_hours,data[pt][Rep].Rep_measured[id,1::12],'--s',color='cornflowerblue')#,marker_array[i],color=color_array[i])   
legend.append(f'c$_p$(t)')
ax.legend(legend,fontsize=18)
ax.set_xlabel('time [hours]')
ax.set_ylabel('Re$_p^{\\mathrm{M}}$')

ax.set_title(f'start location = ({data[pt][Rep].lon[id,0].values:.02f}'+'$^{\\circ}$'+f', {data[pt][Rep].lat[id,0].values:.02f}'+'$^{\\circ}$)',fontsize=18)

fig.savefig('../figures/Rep-in_vs_Rep-measured/trajectory_Rep-measured.pdf')

### Estimate when measured and input particle reynolds number are the same
To estimate when the measured and input reynolds number are the same we use the  
fact that the slip velocity scales with $C(\mathrm{Re}_p^{\mathrm{in}})$, Thus we try   
to predict $\mathrm{Re}_p^{\mathrm{m}}$ using a single reference measurement with   
$\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{in}}$ resulting in $\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{m}}$  
Then we calculate the  $\mathrm{Re}_p^{\mathrm{measured}}$ using:   
$$  \mathrm{Re}_p^{\mathrm{m}}=\frac{\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{m}}  C(\mathrm{Re}_{p,\mathrm{ref}}^{\mathrm{in}})}{C(\mathrm{Re}_{p}^{\mathrm{in}}) }  $$

In [ ]:
# define functions needed to estimate the measured Rep based on a single reference measured
def Rep_measured(Rep_in, Rep_ref_in, Rep_ref_measured):
    return Rep_ref_measured * pcf.factor_drag_white1991(Rep_ref_in)/pcf.factor_drag_white1991(Rep_in)

def Rep_in_measured_root(Rep_in, Rep_ref_in, Rep_ref_measured):
   return Rep_measured(Rep_in, Rep_ref_in, Rep_ref_measured)-Rep_in

In [ ]:

fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))


Replist = [0,200,400,600,800,1000]

Repmeasured = []
solution= []
for Rep in Replist:
    Repm = data['inertial_Rep_constant'][Rep].Rep_measured.isel(obs=slice(12,None)).mean(skipna=True).values
    Repmeasured.append(Repm)
    Rep_sol = fsolve(Rep_in_measured_root, 450, args=(Rep, Repm))[0] 
    solution.append(Rep_sol)

Rep_fit =  np.array(solution[1:]).mean()
ax.plot(pcf.factor_drag_white1991(np.array(Replist)),Repmeasured,'s',color='firebrick',zorder=-2)
r=Repmeasured[2] * pcf.factor_drag_white1991(Replist[2])

Reprange = np.logspace(-1,4,1000)
ax.plot(pcf.factor_drag_white1991(Reprange),r/pcf.factor_drag_white1991(Reprange),'--',zorder=-5,color='firebrick')

ax.plot(pcf.factor_drag_white1991(Rep_fit),Rep_fit,'o',color='grey')
Rep_flex = data['inertial_drag_Rep'][None].Rep_measured.isel(obs=slice(12,None)).mean(skipna=True).values
ax.plot(pcf.factor_drag_white1991(Rep_flex), Rep_flex,'D',color='cornflowerblue')


ax.plot(pcf.factor_drag_white1991(Reprange),Reprange,color='lightgrey',zorder=-10)

ax.set_xlim(1,30)
ax.set_ylim(0,1200)
# # ax.set_ylim(1E1,1E4)
ax.set_xlabel('c(Re$_p^{\\mathrm{In}})$')
ax.set_ylabel('$\\langle\\mathrm{Re}_p^{\\mathrm{M}}\\rangle$')


ax.legend(['measured $\\langle\\mathrm{Re}_p^{\\mathrm{M}}\\rangle$','fitted $\\langle\\mathrm{Re}_p^{\\mathrm{M}}\\rangle$', '$\\langle\\mathrm{Re}_p^{\\mathrm{M}}\\rangle$ = 512','$\\langle\\mathrm{Re}_p^{\\mathrm{M}}\\rangle$ = 348'],fontsize=18,loc=1,frameon=1)
fig.savefig('../figures/Rep-in_vs_Rep-measured/Rep_measured_vs_Rep_in.pdf')
#print values solutions
print(f'c({Rep_fit:.0f}) = {pcf.factor_drag_white1991(Rep_fit):.2f}')
print(f'c({Rep_flex:.0f}) = {pcf.factor_drag_white1991(Rep_flex):.2f}')




### Scatter plots entire domain after 12 h

In [ ]:
# scatter plot for reynolds number 
pt = 'inertial_drag_Rep'
tid =24*12#13 # 24


fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1),projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
sec = float(data[pt][None].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
ax.set_title('c$_p$(t), ' + f'{date}', fontsize=18)
scp = ax.scatter(data[pt][None].lon[::1,tid].T,data[pt][None].lat[::1,tid].T,c=data[pt][None].Rep_measured[::1,tid].T,s=0.6, vmin=0,vmax=2000,cmap='magma',rasterized=True)
# fig.colorbar(scp,label='Re$_p^{\\mathrm{M}}$',fraction=0.025,extend='max')
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0.1, color='gray', alpha=0, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}

# fig.tight_layout()
fig.savefig('../figures/Rep-in_vs_Rep-measured/scatter_plot_measured_Rep_flexible_rep_24h_no_legend.png')

In [ ]:
# movie
pt = 'inertial_drag_Rep'
tid =24*12#13 # 24
Tmax = 48*12
for tid in range(0,Tmax,1):
    fig, ax= plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    ax.coastlines()
    ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
    sec = float(data[pt][None].time[0,tid].values)
    date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
    ax.set_title('flexible Re$_p$, ' + f'{date}', fontsize=18)
    scp = ax.scatter(data[pt][None].lon[::1,tid].T,data[pt][None].lat[::1,tid].T,c=data[pt][None].Rep_measured[::1,tid].T,s=0.6, vmin=0,vmax=2000,cmap='magma',rasterized=True)
    fig.colorbar(scp,label='Re$_p$',fraction=0.025,extend='max')
    ax.set_xlim(-16,10)
    ax.set_ylim(46,61.3)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                linewidth=0.1, color='gray', alpha=0, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 20}
    gl.ylabel_style =  {'size': 20}

    fig.tight_layout()
    fig.savefig(f'../figures/Rep-in_vs_Rep-measured/frames/frame{tid:04d}.png')
    plt.close()

In [ ]:
# scatter plot for reynolds number 
pt = 'inertial_Rep_constant'
tid =24*12#13 # 24
Rep = 400
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1),projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
sec = float(data[pt][Rep].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
ax.set_title(f'c(400)={pcf.factor_drag_white1991(400):.0f}, ' + f'{date}', fontsize=18)
scp = ax.scatter(data[pt][Rep].lon[::1,tid].T,data[pt][Rep].lat[::1,tid].T,c=data[pt][Rep].Rep_measured[::1,tid].T,s=0.6, vmin=0,vmax=2000,cmap='magma',rasterized=True)
# fig.colorbar(scp,label='Re$_p^{\\mathrm{M}}$',fraction=0.025,extend='max')
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0.1, color='gray', alpha=0, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}

fig.savefig('../figures/Rep-in_vs_Rep-measured/scatter_plot_measured_Rep_400_rep_24h_no_legend.png')

In [ ]:
# scatter plot for reynolds number 
pt = 'inertial_Rep_constant'
tid =24*12#13 # 24
Rep = 400
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1),projection=ccrs.PlateCarree())
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
sec = float(data[pt][Rep].time[0,tid].values)
date =  datetime(2023, 9, 1, 0, 0, 0, 0) + timedelta(seconds =sec)
ax.set_title(f'c(400)={pcf.factor_drag_white1991(400):.0f}, ' + f'{date}', fontsize=18)
scp = ax.scatter(data[pt][Rep].lon[1:10:1,tid].T,data[pt][Rep].lat[1:10:1,tid].T,c=data[pt][Rep].Rep_measured[1:10:1,tid].T,s=0.6, vmin=0,vmax=2000,cmap='magma',rasterized=True)
fig.colorbar(scp,label='Re$_p^{\\mathrm{M}}$',fraction=0.025,extend='max')
ax.set_xlim(-16,10)
ax.set_ylim(46,61.3)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0.1, color='gray', alpha=0, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}

fig.savefig('../figures/Rep-in_vs_Rep-measured/scatter_plot_measured_Rep_400_rep_24h_colorbar.pdf')